# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by `model.profile`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm

print(f"Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v3.9.3


In [2]:
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)

In [3]:
# Load the data
returns = pd.read_csv(pm.get_data("SP500.csv"), index_col=0, parse_dates=True)

In [4]:
# Stochastic volatility example
with pm.Model() as model:
    sigma = pm.Exponential("sigma", 1.0 / 0.02, testval=0.1)
    nu = pm.Exponential("nu", 1.0 / 10)
    s = pm.GaussianRandomWalk("s", sigma ** -2, shape=returns.shape[0])
    r = pm.StudentT("r", nu, lam=np.exp(-2 * s), observed=returns["change"])

Then we call the `profile` function and summarize its return values.

In [5]:
# Profiling of the logp call
model.profile(model.logpt).summary()

Function profiling
  Message: /home/amit/Work/OpenSourceDev/pymc3/pymc3/model.py:1192
  Time in 1000 calls to Function.__call__: 2.042763e-01s
  Time in Function.fn.__call__: 1.832466e-01s (89.705%)
  Time in thunks: 1.743519e-01s (85.351%)
  Total compile time: 7.987223e-01s
    Number of Apply nodes: 22
    Theano Optimizer time: 7.478609e-01s
       Theano validate time: 2.320766e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 3.071356e-02s
       Import time 1.030779e-02s
       Node make_thunk time 2.952695e-02s
           Node Elemwise{Composite{Switch(Cast{int8}((GT(Composite{exp((i0 * i1))}(i0, i1), i2) * i3 * GT(inv(sqrt(Composite{exp((i0 * i1))}(i0, i1))), i2))), (((i4 + (i5 * log(((i6 * Composite{exp((i0 * i1))}(i0, i1)) / i7)))) - i8) - (i5 * i9 * log1p(((Composite{exp((i0 * i1))}(i0, i1) * i10) / i7)))), i11)}}(TensorConstant{(1,) of -2.0}, s, TensorConstant{(1,) of 0}, Elemwise{gt,no_inplace}.0, Elemwise{Composite{scalar_gammaln((i0 * i1))}}.0, T

In [6]:
# Profiling of the gradient call dlogp/dx
model.profile(pm.gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /home/amit/Work/OpenSourceDev/pymc3/pymc3/model.py:1192
  Time in 1000 calls to Function.__call__: 2.495768e-01s
  Time in Function.fn.__call__: 2.238679e-01s (89.699%)
  Time in thunks: 2.018890e-01s (80.893%)
  Total compile time: 1.827419e+00s
    Number of Apply nodes: 47
    Theano Optimizer time: 1.549069e+00s
       Theano validate time: 8.029699e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.464304e-01s
       Import time 1.863251e-01s
       Node make_thunk time 2.439311e-01s
           Node Elemwise{Composite{Switch(i0, ((i1 * i2 * i3 * i4) / i5), i6)}}(Elemwise{Composite{Cast{int8}((GT(i0, i1) * i2 * GT(inv(sqrt(i0)), i1)))}}.0, TensorConstant{(1,) of 0.5}, Elemwise{add,no_inplace}.0, Elemwise{Composite{exp((i0 * i1))}}.0, TensorConstant{[1.0435322..54666e-07]}, Elemwise{Add}[(0, 1)].0, TensorConstant{(1,) of 0}) time 3.346968e-02s
           Node Elemwise{switch,no_inplace}(Elemwise{Composite{Cast{int8}(GT(i0, i1))}

In [7]:
%load_ext watermark
%watermark -n -u -v -iv -w

pymc3  3.9.3
numpy  1.18.5
pandas 1.0.5
last updated: Sun Aug 30 2020 

CPython 3.8.3
IPython 7.16.1
watermark 2.0.2
